In [1]:
 import pandas as pd
 import numpy as np

In [9]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Загрузим данные

In [10]:
df = pd.read_csv('gdrive/MyDrive/train.csv')

Выделим классы

In [11]:
for num, name_class in enumerate(df.category.unique()):
  df.loc[df.category==name_class, 'class'] = num

In [ ]:
df

,text,category,class
0,I am still waiting on my card?,card_arrival,0.0
1,What can I do if my card still hasn't arrived ...,card_arrival,0.0
2,I have been waiting over a week. Is the card s...,card_arrival,0.0
3,Can I track my card while it is in the process...,card_arrival,0.0
4,"How do I know if I will get my card, or if it ...",card_arrival,0.0
...,...,...,...
9998,You provide support in what countries?,country_support,76.0
9999,What countries are you supporting?,country_support,76.0
10000,What countries are getting support?,country_support,76.0
10001,Are cards available in the EU?,country_support,76.0


In [16]:
import nlpaug.augmenter.word as naw
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Разделим на обучающую и тестовую выборку

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25, random_state=241)

Проведем Аугментацию данных путем замены слов на синонимы с помощью библиотеки - nlpaug

In [18]:
aug = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=1, aug_max=10, aug_p=0.2, lang='eng', 
                     stopwords=None, tokenizer=None, reverse_tokenizer=None, stopwords_regex=None, force_reload=False, 
                     verbose=0)
 
test_sentence = df.text[0]
result = pd.DataFrame()
for text, label in zip(train.text, train['class']):
  for _ in range(10):
    result = result.append(pd.DataFrame({'text': aug.augment(text), 'class':label}, index=range(0, 1)))

Мержим все что есть и типом train и test, чтобы в тест не попали данные с аугментации

In [19]:
train = train.append(result)
train['type'] = 'train'
test['type'] = 'test'
df = train.append(test)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,category,class,type
9266,Tell me how to renew my new card?,activate_my_card,71.0,train
4236,i can not see my top up,topping_up_by_card,33.0,train
8715,MY transfer has still not appeared.,balance_not_updated_after_bank_transfer,67.0,train
7425,How can I configure what currency my salary is...,receiving_money,56.0,train
4080,do i need to wait for my card before i get pin,get_physical_card,31.0,train
...,...,...,...,...
1670,Help me unblock my account. I entered the PIN...,pin_blocked,13.0,test
782,My cash withdrawal isnt showing on my account,pending_cash_withdrawal,5.0,test
6107,My transfer's stuck on pending.,pending_transfer,46.0,test
5929,I tried to pay with my card and it didn't work,declined_card_payment,45.0,test


Следующий этап состоит в том, чтобы собрать мешок слов из символов

In [20]:
BOS, EOS = ' ', '\n' 

lines = df.text.apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()
lines = [line.lower() for line in lines]

In [21]:
for i in range(len(lines)):
  lines[i] = lines[i].replace('\xa0', '')

In [22]:
lines[:2]

[' tell me how to renew my new card?\n', ' i can not see my top up\n']

In [23]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
tokens = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]
tokens = [char for line in tokens for char in line] 
tokens.append(BOS)
tokens.append(EOS)
tokens = list(set(tokens))
tokens = sorted(tokens)
n_tokens = len(tokens)

In [24]:
print('shape of tokens = %s'%len(tokens))

shape of tokens = 58


In [25]:
#Зададим каждому символу, свой порядковый номер
token_to_id = {char: t for t, char in enumerate(tokens)}

Функция для перевода текстовых строчек в мешок слов

In [ ]:
def to_matrix(lines, max_len=None, pad=token_to_id[EOS], dtype=np.int64):
    max_len = max_len or max(map(len, lines))
    lines_ix = np.full([len(lines), max_len], pad, dtype=dtype)
    for i in range(len(lines)):
        line_ix = list(map(token_to_id.get, lines[i][:max_len]))
        lines_ix[i, :len(line_ix)] = line_ix
    return lines_ix

In [ ]:
to_matrix(lines[:2])

array([[ 1, 36,  1, 46, 32, 32,  1, 28,  1, 31, 48, 43, 39, 36, 30, 28,
        47, 32,  1, 43, 28, 52, 40, 32, 41, 47, 12,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 50, 35, 52,  1, 31, 42,  1, 36,  1, 35, 28, 49, 32,  1, 28,
         1, 30, 35, 28, 45, 34, 32,  1, 33, 42, 45,  1, 28, 41,  1, 28,
        47, 40,  1, 50, 36, 47, 35, 31, 45, 28, 50, 28, 39, 27,  1, 36,
         1, 47, 35, 42, 48, 34, 35, 47,  1, 47, 35, 32, 46, 32,  1, 50,
        32, 45, 32,  1, 33, 45, 32, 32, 27,  0]])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
n_class = len(df['class'].unique())

In [ ]:
class ClassifierRNN(nn.Module):
    def __init__(self, n_tokens=n_tokens, emb_size=32, hid_size=256, n_class=n_class):
       
        super().__init__() 

        self.emb = nn.Embedding(num_embeddings=n_tokens, embedding_dim=emb_size)
        self.LSTM = nn.LSTM(input_size=emb_size, hidden_size=hid_size, batch_first=True, num_layers=2)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(hid_size, hid_size)
        self.linear2 = nn.Linear(hid_size, hid_size)
        self.linear3 = nn.Linear(hid_size, n_class)
        self.softmax = nn.Softmax()
        
    
    def __call__(self, input_ix):
        x = self.emb(input_ix)
        x = self.LSTM(x)[0][:,-1]
        x = self.relu(self.linear(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

Попытка сделать RNN сеть не увенчалась успехом, плохой результат получился

In [ ]:
class ClassifierCNN(nn.Module):
    def __init__(self, n_tokens=len(tokens), emb_size=64, hid_size=128, n_class=n_class):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings=n_tokens, embedding_dim=emb_size)
        self.cnn1 = nn.Sequential(
                            nn.Conv1d(emb_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU()
                            )
        self.cnn2 = nn.Sequential(
                            nn.Conv1d(hid_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU()
                            )
        self.cnn3 = nn.Sequential(
                            nn.Conv1d(hid_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU(),
                            nn.AdaptiveMaxPool1d(output_size=1)
                            )
        self.linear = nn.Sequential(
                            nn.Linear(hid_size, 4*hid_size),
                            nn.ReLU(),
                            nn.Linear(hid_size*4, hid_size*4),
                            nn.ReLU(),
                            nn.Linear(4*hid_size, n_class)
                            )
        
    def __call__(self, input_ix):
        x = self.emb(input_ix).transpose(1, 2)
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x).transpose(1, 2)
        return self.linear(x).squeeze()
        

Простая сеть с 3-мя сверточными слоями

In [ ]:
class ClassifierCNN(nn.Module):
    def __init__(self, n_tokens=len(tokens), emb_size=64, hid_size=128, n_class=n_class):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings=n_tokens, embedding_dim=emb_size)
        self.cnn1 = nn.Sequential(
                            nn.Conv1d(emb_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU(),
                            nn.BatchNorm1d(hid_size)
                            )
        self.cnn2 = nn.Sequential(
                            nn.Conv1d(hid_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU(),
                            nn.BatchNorm1d(hid_size)
                            )
        self.cnn3 = nn.Sequential(
                            nn.Conv1d(hid_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU(),
                            nn.BatchNorm1d(hid_size)
                            )
        self.cnn4 = nn.Sequential(
                            nn.Conv1d(hid_size, hid_size, kernel_size=3),
                            nn.Dropout(p=0.25),
                            nn.ReLU(),
                            nn.AdaptiveMaxPool1d(output_size=1),
                            nn.BatchNorm1d(hid_size)
                            )
        self.linear = nn.Sequential(
                            nn.Linear(hid_size, 4*hid_size),
                            nn.ReLU(),
                            nn.Linear(hid_size*4, hid_size*4),
                            nn.ReLU(),
                            nn.Dropout(p=0.25),
                            nn.Linear(hid_size*4, hid_size*4),
                            nn.ReLU(),
                            nn.Linear(4*hid_size, n_class)
                            )
        
    def __call__(self, input_ix):
        x = self.emb(input_ix).transpose(1, 2)
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.cnn4(x).transpose(1, 2)
        return self.linear(x).squeeze()
        

Немного улучшаем

In [ ]:
df['lines'] = lines
df.head(2)

,text,category,class,type,lines
9266,Tell me how to renew my new card?,activate_my_card,71.0,train,tell me how to renew my new card?\n
4236,i can not see my top up,topping_up_by_card,33.0,train,i can not see my top up\n


In [ ]:
train_text = df.loc[df.type=='train', 'lines'].to_list()
test_text = df.loc[df.type=='test', 'lines'].to_list()
train_labels = df.loc[df.type=='train', 'class'].to_list()
test_labels = df.loc[df.type=='test', 'class'].to_list()

In [ ]:
#переводим в тензора
train_matrix = torch.tensor(to_matrix(train_text))

test_matrix = torch.tensor(to_matrix(test_text))

In [ ]:
train_labels = torch.tensor(train_labels)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [ ]:
batch_size = 256
train_data = TensorDataset(train_matrix, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

Т.к метки несбалансированные, подсчитаем веса для каждой метки и передадим их, как аргумет в функцию потерь - кросс_энтропию

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', np.unique(train_labels), train_labels)


In [ ]:
device = torch.device('cuda')
weights= torch.tensor(class_weights,dtype=torch.float)

weights = weights.to(device)


Обучение

In [ ]:
from tqdm.notebook import tqdm
from torch.optim import lr_scheduler
model = ClassifierCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"epoch: {epoch}")
    model.train()
    for  i, batch in enumerate(tqdm(train_dataloader)):
        batch = [r.to(device) for r in batch]
        sent_id,  labels = batch
        optimizer.zero_grad()
        pred = model(sent_id)
        labels = labels.long()
        #target = batch['class'].long()
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        if i % 20 == 0:
          print('loss = %f'%loss.item())

epoch: 0


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 4.341788
loss = 4.334428
loss = 4.308545
loss = 4.292790
loss = 4.274245
loss = 4.310888
loss = 4.235419
loss = 4.136908
loss = 4.085417
loss = 3.910193
loss = 3.814699
loss = 3.816343
loss = 3.638532
loss = 3.636296
loss = 3.551883
loss = 3.350270
loss = 3.360361
epoch: 1


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 3.182056
loss = 3.090386
loss = 3.249537
loss = 3.284030
loss = 3.126810
loss = 3.083251
loss = 3.095666
loss = 2.865837
loss = 2.832069
loss = 2.892827
loss = 2.824306
loss = 2.712342
loss = 2.615083
loss = 2.790845
loss = 2.536636
loss = 2.570494
loss = 2.648888
epoch: 2


...

loss = 1.295890
loss = 1.176460
loss = 1.180734
loss = 1.138168
loss = 1.314819
loss = 1.132066
loss = 1.274629
loss = 1.127044
loss = 1.087569
loss = 1.118252
loss = 1.233875
loss = 1.095969
loss = 1.017406
loss = 0.939166
loss = 1.158645
loss = 1.155480
loss = 1.058894
epoch: 8


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 1.146747
loss = 1.123288
loss = 1.210436
loss = 1.185087
loss = 1.008755
loss = 1.122216
loss = 1.024320
loss = 0.959082
loss = 1.010312
loss = 1.034966
loss = 0.980612
loss = 1.111919
loss = 1.067711
loss = 0.884091
loss = 0.962864
loss = 1.131087
loss = 1.078777
epoch: 9


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 1.037803
loss = 1.159962
loss = 1.119490
loss = 1.012737
loss = 0.927251
loss = 1.233433
loss = 1.032953
loss = 0.996858
loss = 0.899910
loss = 0.976690
loss = 0.950469
loss = 0.921913
loss = 1.089092
loss = 0.953282
loss = 1.039637
loss = 0.839102
loss = 1.033888


In [ ]:
from sklearn.metrics import f1_score
def test_score(model, test_matrix, device):
  f1 = 0

  model.eval()
  with torch.no_grad():
    pred = model(test_matrix.to(device))
    pred = torch.argmax(pred, dim=1).cpu().detach().numpy()
    f1 = f1_score(pred, test_labels, pred, average='macro')
  return f1

In [ ]:
from tqdm.notebook import tqdm
from torch.optim import lr_scheduler
model = ClassifierCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=400, gamma=0.1)
EPOCHS = 15
f1 = 0
params_model = 0
for epoch in range(EPOCHS):
    print(f"epoch: {epoch}")
    model.train()
    for  i, batch in enumerate(tqdm(train_dataloader)):
        batch = [r.to(device) for r in batch]
        sent_id,  labels = batch
        optimizer.zero_grad()
        pred = model(sent_id)
        labels = labels.long()
        #target = batch['class'].long()
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        if test_score(model, test_matrix, device)>f1:
          f1 = test_score(model, test_matrix, device)
          params_model = model.state_dict()
        if i % 20 == 0:
          print('loss = %f'%loss.item(), 'f1_score test = ',f1 )

epoch: 0


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 4.333051 f1_score test =  0.022924901185770754
loss = 4.340577 f1_score test =  0.028379976350019712
loss = 4.330057 f1_score test =  0.028379976350019712
loss = 4.315484 f1_score test =  0.03148366784730422
loss = 4.333817 f1_score test =  0.03148366784730422
loss = 4.306513 f1_score test =  0.03148366784730422
loss = 4.303718 f1_score test =  0.03148366784730422
loss = 4.255809 f1_score test =  0.03714548493888704
loss = 4.241342 f1_score test =  0.06785494711392036
loss = 4.056478 f1_score test =  0.07029318281462509
loss = 3.909437 f1_score test =  0.07029318281462509
loss = 3.931465 f1_score test =  0.07612423107954289
loss = 3.803548 f1_score test =  0.08484851004657169
loss = 3.796975 f1_score test =  0.10953963712647456
loss = 3.563515 f1_score test =  0.129126056432543
loss = 3.506716 f1_score test =  0.14294784479666917
loss = 3.429619 f1_score test =  0.17693112712247328
epoch: 1


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 3.807784 f1_score test =  0.17693112712247328
loss = 3.627548 f1_score test =  0.17693112712247328
loss = 3.561411 f1_score test =  0.17693112712247328
loss = 3.450225 f1_score test =  0.17693112712247328
loss = 3.399137 f1_score test =  0.18096971935267955
loss = 3.343088 f1_score test =  0.18469601181704737
loss = 3.228672 f1_score test =  0.18521022415658053
loss = 3.095377 f1_score test =  0.18876770576263258
loss = 3.170963 f1_score test =  0.21161039869554685
loss = 3.036689 f1_score test =  0.21773232702057818
loss = 3.154443 f1_score test =  0.22206235067561275
loss = 3.081253 f1_score test =  0.2395267948425143
loss = 2.938586 f1_score test =  0.24056825046033223
loss = 2.805140 f1_score test =  0.2623518054681758
loss = 2.905184 f1_score test =  0.2683226039825396
loss = 2.706948 f1_score test =  0.283183532515984
loss = 2.718599 f1_score test =  0.30522775760979265
epoch: 2


...

loss = 3.575835 f1_score test =  0.8136282538840737
loss = 0.318610 f1_score test =  0.8136282538840737
loss = 0.443527 f1_score test =  0.8136282538840737
loss = 0.357025 f1_score test =  0.8136282538840737
loss = 0.357926 f1_score test =  0.8136282538840737
loss = 0.318624 f1_score test =  0.8157704299070311
loss = 0.415966 f1_score test =  0.8157704299070311
loss = 0.364197 f1_score test =  0.817218599579626
loss = 0.394739 f1_score test =  0.817218599579626
loss = 0.233619 f1_score test =  0.817218599579626
loss = 0.463955 f1_score test =  0.817218599579626
loss = 0.320580 f1_score test =  0.817218599579626
loss = 0.383343 f1_score test =  0.817218599579626
loss = 0.421067 f1_score test =  0.8198972238740403
loss = 0.327417 f1_score test =  0.8198972238740403
loss = 0.418147 f1_score test =  0.8198972238740403
loss = 0.428344 f1_score test =  0.8198972238740403
epoch: 13


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 3.569323 f1_score test =  0.8198972238740403
loss = 0.343627 f1_score test =  0.8198972238740403
loss = 0.299583 f1_score test =  0.8198972238740403
loss = 0.360819 f1_score test =  0.8202880157539338
loss = 0.311093 f1_score test =  0.8202880157539338
loss = 0.326485 f1_score test =  0.8202880157539338
loss = 0.355652 f1_score test =  0.8202880157539338
loss = 0.356214 f1_score test =  0.8208707249628232
loss = 0.279828 f1_score test =  0.8208707249628232
loss = 0.315582 f1_score test =  0.8226709402364537
loss = 0.325685 f1_score test =  0.8226709402364537
loss = 0.441789 f1_score test =  0.8226709402364537
loss = 0.324359 f1_score test =  0.8227125275923517
loss = 0.395139 f1_score test =  0.8227125275923517
loss = 0.374317 f1_score test =  0.8227125275923517
loss = 0.275440 f1_score test =  0.8227125275923517
loss = 0.302723 f1_score test =  0.8227125275923517
epoch: 14


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 3.583187 f1_score test =  0.8227125275923517
loss = 0.222897 f1_score test =  0.8227125275923517
loss = 0.276702 f1_score test =  0.8227125275923517
loss = 0.338383 f1_score test =  0.8227125275923517
loss = 0.286983 f1_score test =  0.8236797605785647
loss = 0.352642 f1_score test =  0.8236797605785647
loss = 0.320782 f1_score test =  0.8253582445297397
loss = 0.280404 f1_score test =  0.8253582445297397
loss = 0.365577 f1_score test =  0.8253582445297397
loss = 0.319370 f1_score test =  0.8253582445297397
loss = 0.285768 f1_score test =  0.8253582445297397
loss = 0.245819 f1_score test =  0.8253582445297397
loss = 0.347355 f1_score test =  0.8253582445297397
loss = 0.330593 f1_score test =  0.8253582445297397
loss = 0.318129 f1_score test =  0.8253582445297397
loss = 0.253024 f1_score test =  0.8253582445297397
loss = 0.257543 f1_score test =  0.8272277243625022


In [ ]:
import os
torch.save(params_model, os.path.join('gdrive/MyDrive/', 'best_model'))

Проверка модели с SelfAdjDiceLoss 

In [ ]:
class SelfAdjDiceLoss(torch.nn.Module):

    def __init__(self, alpha: float = 1.0, gamma: float = 1.0, reduction: str = "mean") -> None:
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        probs = torch.softmax(logits, dim=1)
        probs = torch.gather(probs, dim=1, index=targets.unsqueeze(1))

        probs_with_factor = ((1 - probs) ** self.alpha) * probs
        loss = 1 - (2 * probs_with_factor + self.gamma) / (probs_with_factor + 1 + self.gamma)

        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        elif self.reduction == "none" or self.reduction is None:
            return loss
        else:
            raise NotImplementedError(f"Reduction `{self.reduction}` is not supported.")

In [ ]:
from tqdm.notebook import tqdm
from torch.optim import lr_scheduler
model = ClassifierCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=400, gamma=0.1)
EPOCHS = 15
f1 = 0
params_model = 0
for epoch in range(EPOCHS):
    print(f"epoch: {epoch}")
    model.train()
    for  i, batch in enumerate(tqdm(train_dataloader)):
        batch = [r.to(device) for r in batch]
        sent_id,  labels = batch
        optimizer.zero_grad()
        pred = model(sent_id)
        labels = labels.long()
        #target = batch['class'].long()
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        f1_tmp = test_score(model, test_matrix, device)
        if f1_tmp>f1:
          f1 = f1_tmp
          params_model = model.state_dict()
        if i % 20 == 0:
          print('loss = %f'%loss.item(), 'f1_score test = ', f1_tmp)

epoch: 0


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 4.344694 f1_score test =  0.02448657187993681
loss = 4.340944 f1_score test =  0.02448657187993681
loss = 4.335081 f1_score test =  0.030442506380471534
loss = 4.317577 f1_score test =  0.022924901185770754
loss = 4.312341 f1_score test =  0.022924901185770754
loss = 4.283731 f1_score test =  0.022924901185770754
loss = 4.286858 f1_score test =  0.022924901185770754
loss = 4.270424 f1_score test =  0.04503894816576089
loss = 4.201200 f1_score test =  0.054843555818678294
loss = 4.053119 f1_score test =  0.06728562332163933
loss = 3.999486 f1_score test =  0.04358608192657985
loss = 3.964964 f1_score test =  0.06489640787508695
loss = 3.902173 f1_score test =  0.07323194949495875
loss = 3.798434 f1_score test =  0.08455297987978956
loss = 3.821606 f1_score test =  0.09547246815090615
loss = 3.687636 f1_score test =  0.1070552172275461
loss = 3.674948 f1_score test =  0.12693617361499132
epoch: 1


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 4.089890 f1_score test =  0.03976312406008121
loss = 3.780951 f1_score test =  0.10108022997953953
loss = 3.513255 f1_score test =  0.15518148476325036
loss = 3.382892 f1_score test =  0.17075319535983954
loss = 3.347241 f1_score test =  0.21583700155755906
loss = 3.328513 f1_score test =  0.1800862803971159
loss = 3.130100 f1_score test =  0.22333680845299905
loss = 3.017849 f1_score test =  0.2503979184954317
loss = 3.075322 f1_score test =  0.25768610051047597
loss = 2.947673 f1_score test =  0.26772933586186604
loss = 2.776262 f1_score test =  0.29160209459519637
loss = 2.713614 f1_score test =  0.2777351926617068
loss = 2.605471 f1_score test =  0.2965042309691886
loss = 2.636892 f1_score test =  0.31201821755432707
loss = 2.539486 f1_score test =  0.32957006748672146
loss = 2.615698 f1_score test =  0.3370173272372037
loss = 2.230172 f1_score test =  0.34757784581857865
epoch: 2


...

loss = 3.464351 f1_score test =  0.7934575328627635
loss = 0.339922 f1_score test =  0.798452528187645
loss = 0.443840 f1_score test =  0.8110090755202326
loss = 0.318464 f1_score test =  0.8088856554066459
loss = 0.399199 f1_score test =  0.8089658897591595
loss = 0.377918 f1_score test =  0.814912852437234
loss = 0.375582 f1_score test =  0.8202225015661401
loss = 0.375551 f1_score test =  0.8153170672684724
loss = 0.428398 f1_score test =  0.8082719279069469
loss = 0.311597 f1_score test =  0.8140191027077762
loss = 0.380004 f1_score test =  0.8104699427590427
loss = 0.392910 f1_score test =  0.8142878399760126
loss = 0.370969 f1_score test =  0.8149019923467593
loss = 0.348088 f1_score test =  0.8072396189804305
loss = 0.329974 f1_score test =  0.8180904863551768
loss = 0.321437 f1_score test =  0.8202542971220085
loss = 0.365932 f1_score test =  0.8179408890826835
epoch: 13


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 3.388175 f1_score test =  0.8029036315598538
loss = 0.319861 f1_score test =  0.8105850684646768
loss = 0.261368 f1_score test =  0.8163123311253251
loss = 0.287383 f1_score test =  0.8093850198360231
loss = 0.254900 f1_score test =  0.8167804892355119
loss = 0.269838 f1_score test =  0.8087530556620461
loss = 0.348378 f1_score test =  0.8137593273452564
loss = 0.363147 f1_score test =  0.8119435255247883
loss = 0.418221 f1_score test =  0.8255463486368007
loss = 0.401102 f1_score test =  0.8172350448812238
loss = 0.281728 f1_score test =  0.8152209542896368
loss = 0.315673 f1_score test =  0.8153781382116447
loss = 0.319890 f1_score test =  0.8217079549370292
loss = 0.274401 f1_score test =  0.8163870636270549
loss = 0.277367 f1_score test =  0.8152532934301442
loss = 0.337910 f1_score test =  0.8147864407940613
loss = 0.342172 f1_score test =  0.8183610724050515
epoch: 14


  0%|          | 0/323 [00:00<?, ?it/s]

loss = 3.710405 f1_score test =  0.7935447762105456
loss = 0.268102 f1_score test =  0.8223030928029146
loss = 0.295395 f1_score test =  0.8153810315807836
loss = 0.251980 f1_score test =  0.8161008140370621
loss = 0.348536 f1_score test =  0.8236929480298286
loss = 0.296359 f1_score test =  0.8176507664379907
loss = 0.379505 f1_score test =  0.8130335458565513
loss = 0.262314 f1_score test =  0.8245846633361984
loss = 0.241769 f1_score test =  0.8126664150231078
loss = 0.238844 f1_score test =  0.8168662320316952
loss = 0.349524 f1_score test =  0.8218681545817035
loss = 0.279137 f1_score test =  0.8203770992217918
loss = 0.339232 f1_score test =  0.8139177341690035
loss = 0.249672 f1_score test =  0.8159514381652749
loss = 0.266252 f1_score test =  0.8136234917864512
loss = 0.318513 f1_score test =  0.8138429369072003
loss = 0.230122 f1_score test =  0.812831771660833


In [ ]:
f1

0.8281057443793844